In [ ]:
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.constants import SPATIAL, NODATAINT, MAX, SQUARE, EXACT, ZOOM
from geopyspark.geotrellis.rdd import TiledRasterRDD
from geopyspark.geotrellis.geotiff_rdd import get
from geopyspark.geotrellis.render import PngRDD
from geonotebook.wrappers import RddRasterData, VectorData

In [ ]:
geopysc = GeoPyContext(appName="testing", master="local[*]")

In [ ]:
M.set_center(-122.1, 37.75, 10)

In [ ]:
import fiona
from shapely.geometry import *

In [ ]:
!curl -o /tmp/bart.geojson https://s3.amazonaws.com/geopyspark-demo/bayarea/bart.geojson
!curl -o /tmp/school.geojson https://s3.amazonaws.com/geopyspark-demo/bayarea/school.geojson
!curl -o /tmp/parks.geojson https://s3.amazonaws.com/geopyspark-demo/bayarea/parks.geojson

In [ ]:
with fiona.open("/tmp/bart.geojson") as source:
    bart_crs = source.crs['init']
    bart = MultiPoint([shape(f['geometry']) for f in source])

with fiona.open("/tmp/school.geojson") as source:
    schools_crs = source.crs['init']
    schools = MultiPoint([shape(f['geometry']) for f in source])

with fiona.open("/tmp/parks.geojson") as source:
    parks_crs = source.crs['init']
    parks = MultiPolygon([shape(f['geometry']) for f in source])

In [ ]:
bart_layer = TiledRasterRDD.euclidean_distance(geopysc, bart, bart_crs, 12)
schools_layer = TiledRasterRDD.euclidean_distance(geopysc, schools, schools_crs, 12)
parks_layer = TiledRasterRDD.euclidean_distance(geopysc, parks, parks_crs, 12)

In [ ]:
from pyspark.storagelevel import StorageLevel

In [ ]:
bart_layer.persist(StorageLevel.MEMORY_AND_DISK)
schools_layer.persist(StorageLevel.MEMORY_AND_DISK)
parks_layer.persist(StorageLevel.MEMORY_AND_DISK)

In [ ]:
weighted_layer = -1 * bart_layer - schools_layer + 3 * parks_layer

In [ ]:
weighted_layer.persist(StorageLevel.MEMORY_AND_DISK)

In [ ]:
png_layer = PngRDD.makePyramid(weighted_layer, "viridis", end_zoom=8)

In [ ]:
png_layer.cache()

In [ ]:
M.add_layer(RddRasterData(png_layer, name="Weighted"))
M.add_layer(VectorData("/tmp/bart.geojson"), name="BART stops")
M.add_layer(VectorData("/tmp/parks.geojson"), name="Parks")

In [ ]:
M.add_layer(VectorData("/tmp/bart.geojson"), name="BART stops", colors=[0xff0000])
M.add_layer(VectorData("/tmp/parks.geojson"), name="Parks", colors=[0xffff00])

In [ ]:
M.layers

In [ ]:
M.remove_layer(M.layers[2])
M.remove_layer(M.layers[1])
M.remove_layer(M.layers[0])